In [ ]:
import os,CustomAttention
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# parse examples
def decode(example):
    features = {
        'ecg': tf.io.FixedLenFeature([4098], tf.float32),
        'label': tf.io.FixedLenFeature([16], tf.float32),
    }
    feature_dict = tf.io.parse_single_example(example,features)
    ecg = tf.reshape(feature_dict['ecg'],[4098,1])
    label = tf.reshape(feature_dict['label'],[8,2])
    return ecg,label

In [ ]:
# initialization
sample_size = 2000000
LEARNING_RATE = 0.01
EPOCH_NUM = 50
BATCH_SIZE = 256
BUFFER_SIZE = 48000
NUM_PARALEEL_CALLS = 8
NUM_PARALEEL_READS = 9

In [ ]:
temp= [i for i in os.walk('/tmpdata/train/')][0][2] # get all file path
train_files = ['/tmpdata/train/'+file for file in temp]
print(train_files)

temp = [i for i in os.walk('/tmpdata/val/')][0][2] # get all file path
val_files = ['/tmpdata/val/'+file for file in temp]
print(val_files)

In [ ]:
dataset_train = tf.data.TFRecordDataset(train_files,num_parallel_reads=NUM_PARALEEL_READS) # load train dataset

dataset_train = dataset_train.shuffle(buffer_size = BUFFER_SIZE).repeat() # dataset preprocessing
dataset_train = dataset_train.map(decode,num_parallel_calls = NUM_PARALEEL_CALLS)
dataset_train = dataset_train.batch(batch_size = BATCH_SIZE)
dataset_train = dataset_train.prefetch(4)

In [ ]:
dataset_val = tf.data.TFRecordDataset(val_files) # load val dataset

dataset_val = dataset_val.map(decode,num_parallel_calls = NUM_PARALEEL_CALLS) # dataset preprocessing
dataset_val = dataset_val.batch(batch_size = 256, drop_remainder=True)

In [ ]:
from tensorflow.keras import backend as K

def yolo_loss(y_true, y_pred):
    true_x = y_true[...,1]
    pred_x = y_pred[...,1]
    pred_prob = y_pred[...,0]
    object_mask = y_true[...,0]
    
    object_loss = 3*object_mask*K.square(1-pred_prob)
    no_object_loss = 2*(1-object_mask)*K.square(0-pred_prob)
    x_loss = 5*object_mask*K.square(true_x-pred_x)
    loss = object_loss + no_object_loss + x_loss
    return loss

In [ ]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.9,patience=3, mode = 'min', cooldown = 1, min_lr=0.0005),

    ModelCheckpoint(filepath = "../../model/new_yolo_model/model_{epoch:03d}.h5",
                             monitor = 'val_loss', mode = 'min'),
]

In [ ]:
yolo = load_model("../../model/new_yolo_model/model_yolo.h5")

yolo.compile(
    loss=yolo_loss,
    optimizer=keras.optimizers.Adam(learning_rate = LEARNING_RATE)
)
history = yolo.fit(dataset_train,epochs=EPOCH_NUM, steps_per_epoch = sample_size/BATCH_SIZE, 
                    validation_data=dataset_val,verbose = 1,callbacks=callbacks)

In [ ]:
def plot_metric(rate,size,history):
    metrics = [key for key in history.history.keys()][:5] # plot all metric information

    for metric in metrics:
        fig = plt.figure(figsize=(10,3))
        plt.plot(history.history[metric][5:])
        plt.plot(history.history[metric][5:])
        plt.title(metric+ ' rate: '+str(rate)+' size: '+str(size))
        plt.ylabel(metric)
        plt.xlabel('epoch')

        plt.legend(['train', 'val'], loc='upper left')
        plt.show()

plot_metric(0.01,256,history)

In [ ]:
#model.save('../../model/yolo_model.h5')

In [1]:
a = ""
a += "a"
a

'a'